In [ ]:
from flask import Flask
#这是其中一种导入config的方法, 虽说不是很正规, 但也不是不行
# from config import DEBUG


app=Flask(__name__)
app.config.from_object('config')# 导入配置文件, 括号里面是模块路径
#这个app.route内部也就是直接调用了app.add_url_rule() 来实现的
@app.route('/hello/')#加斜杠可以访问/hello本质上就是重定向,就是把/hello 重定向到/hello/
#                     # url唯一访问原理, 如果/hello 和/hello/ 搜索两次的话跟SEO有关
def hello():
    #base on class既插试图
    return 'hello world!!'



# def hello():
#     #base on class既插试图
#     return 'hello world!!'
# app.add_url_rule('/hello',view_func=hello)
#这种更适合用于基于类的视图(既插试图)


app.run(host='0.0.0.0',debug=app.config['DEBUG'],port='5000')#全是0 表示可以接受外网访问



In [ ]:
from flask import Flask, make_response
# from config import DEBUG


app=Flask(__name__)
app.config.from_object('config')
@app.route('/hello/')
def hello():#本质上来说, 这个hello函数返回的是用response封装的一个对象, 这个对象时可以修改的
    #默认的'content-type'='text/html'
    headers={
        'content-type':'text/plain',
#         'content-type':'application/json', 当前端是小程序的时候, 我们需要返回json对象, 用这个
        'location':'www.bing.com'# 这个是用于301 重定向的
    }
    response=make_response('<html></html>',404)#状态码本身不会改变结果, 只不过是通知浏览器状态而已
    response.headers=headers
    return response

@app.route('/hello/')
def hello():
    headers={
        'content-type':'text/plain',
#         'content-type':'application/json', 当前端是小程序的时候, 我们需要返回json对象, 用这个
        'location':'www.bing.com'# 这个是用于301 重定向的
    }
    return 'html', 301, headers# 这实际上是个元祖tuple
#  也可以直接用这个, 就不用new 一个response了
if __name__=='__main__':# 之所以用这个, 是因为这个只在开发环境中使用, 而生产环境中, 我们用专门的nginx+uwsgi
    app.run(host='0.0.0.0',debug=app.config['DEBUG'],port='5000')#全是0 表示可以接受外网访问
    #也就是说, 在生产环境下, 这个根本就执行不了. (因为是nginx+uwsgi执行的) 
    '''
    这部分还是挺重要的
    '''


In [ ]:
from flask import Flask, make_response
# from config import DEBUG
from helper import is_isbn_or_keyword
app=Flask(__name__)
app.config.from_object('config')


@app.route('/book/search/<q>/<page>')
def search(q,page):
    """
        q: normal keyword and isbn
        page
    """
    isbn_or_keyword=is_isbn_or_keyword(q)
    pass


if __name__=='__main__':
    app.run(host='127.0.0.1',debug=app.config['DEBUG'],port='5000')
    
"""
    视图函数(就是这个), 应该尽量的简介, 这样便于维护和增加功能.
    在读源代码的时候也是一样, 不用指望这一口吃个胖子, 慢慢来, 部分功能知道怎么回事就可以了
    不需要弄懂, 梳理一部分后,再回头抠细节.
"""

In [ ]:
import requests

#urllib, requests, 两种http请求发送方法,requests 更适合人类使用
class HTTP:
    def get(self,url,return_json=True):
        r=requests.get(url)
        #restful needed!
        if r.status_code!=200:#这个可以作为正常流程下的特例处理情况(比较不错哦)
            return {} if return_json else ''
        return r.json() if return_json else r.text#用Python三元表达式可以有效降低代码行数
"""
 这是用于接收解读API返回信息的, 用一个类来单独存放HTTP相关功能. 
 代码要尽量简洁易懂.
 
"""

In [ ]:
import json

from flask import Flask, make_response
# from config import DEBUG
from flask.json import jsonify

from helper import is_isbn_or_keyword
from sharebook_book import SharebookBook
app=Flask(__name__)
app.config.from_object('config')
app.config['JSON_AS_ASCII']=False#用来设置正常显示中文的, 否则用默认的话, 会显示编码错误


@app.route('/book/search/<q>/<page>')
def search(q,page):
    """
        q: normal keyword and isbn
        page
    """
    isbn_or_keyword=is_isbn_or_keyword(q)
    if isbn_or_keyword == 'isbn':
        result=SharebookBook.search_by_isbn(q)
    else:
        result=SharebookBook.search_by_keyword(q)#dict 需要序列化(json化)
    return jsonify(result)#这个就是API啦~ 数据的json化就是API
    # return json.dumps(result),200,{'Content-Type':'application/json'}


if __name__=='__main__':
    app.run(host='127.0.0.1',debug=app.config['DEBUG'],port='5000')
    
    """
    讲的主要什么是API和序列化字典
    """

In [ ]:
from flask import jsonify, request, render_template, flash

from app.forms.book import SearchForm
from app.view_models.book import BookViewModel, BookCollection
from app.web.blueprint import web
from app.libs.helper import is_isbn_or_keyword
from app.spider.sharebook_book import SharebookBook

import json

@web.route('/test')
def test():
    r={
        'name':'zhazha??',
        'age':18,
    }
    flash('hello! zhazha!',category='warnning!')
    flash('This is the second msg!',category='error!')
    return render_template('test.html',data=r)


@web.route('/book/search')
def search():
    """
        q: normal keyword and isbn
        page
        ?q=xxx&page=1
        Request/Response class!
        Request记录了HTTP的相关请求信息
    """
    #在用之前, 需要验证合法性!
    # q=request.args['q']#request.args类似字典,是字典的子类, 不可变字典,可以转化成普通的可变字典 a=request.args.to_dict()
    # # q 至少需要有一个字符, 对长度有限制
    # page=request.args['page']
    # 页数需要是正整数, 有最大值限制
    form =SearchForm(request.args)
    books=BookCollection()
    #验证层
    if form.validate():
        q=form.q.data.strip()#keyword or isbn
        page=form.page.data#从form里面取得的值会经过判断~触发default=1
        isbn_or_keyword=is_isbn_or_keyword(q)
        sharebook_book = SharebookBook()

        if isbn_or_keyword == 'isbn':
            sharebook_book.search_by_isbn(q)
            # result=SharebookBook.search_by_isbn(q)
            # result=BookViewModel.package_single(result,q)
        else:
            # result=SharebookBook.search_by_keyword(q,page)#dict 需要序列化(json化)
            # result=BookViewModel.package_collection(result,q)
            sharebook_book.search_by_keyword(q,page)

            # print('test', result['books'])
        # return jsonify(result)#这个就是API啦~ 数据的json化就是API
        books.fill(sharebook_book,q)
        # return jsonify(books.__dict__)# jsoify 需要的是字典. 魔术方法的__dict__可以把类里的属性转化成字典. 但该种方法仅仅可以转换简单属性, 比如说在本例中, book.books==[], 这种就无法转换了
        return json.dumps(books, default=lambda x: x.__dict__)
    else:
        return jsonify(form.errors)
